In [1]:
import artm
import os

name = 'booksX2'
model_path = 'models/'+name+'/'
# список приложений / описание приложений ф фомате wv ( | text ) / название папки для artm-батчей
wv_file  = name+'.wv'

batch_folder = 'batches'
# максимальное кол-во записей в одном батче
batch_size = 1000

path = model_path+batch_folder
if os.path.exists(path):
    # создание класса, работающего с батчами (уже заполненного)
    batch_vectorizer = artm.BatchVectorizer(data_path=model_path+batch_folder, data_format='batches')
else:
    # создание класса, работающего с батчами и заполнение ими папки folder+'-'+str(batch_size)
    batch_vectorizer = artm.BatchVectorizer(data_path=model_path+wv_file,
                                            data_format="vowpal_wabbit",
                                            target_folder=path,
                                            batch_size=batch_size)

In [2]:
import os
def doInitDic(dic_name,min_df=2):
    global dictionary
    dn = dic_name+'_'+str(min_df)+'.dict'
    if os.path.isfile(dn):
        dictionary = artm.Dictionary('dictionary')
        dictionary.load(dn)
        print('loaded dict: '+dn+'!')
    else:
        dictionary = artm.Dictionary('dictionary')
        dictionary.gather(batch_vectorizer.data_path)
        if min_df < 0:
            dictionary.filter(min_df_rate=-min_df,max_df_rate=0.3)
        else:
            dictionary.filter(min_df=min_df,max_df_rate=0.3)
        dictionary.save(dn[:-5])
        print('saved dict: '+dn+'!')

# инициализация модели и присоединение к ней словаря
def doInit(T_init=300, reuse_theta=False, cache_theta=False):
    global T, model_artm, dictionary, topic_names
    T = T_init # количество тем
    topic_names = ["sbj"+str(i) for i in range(T)]
    model_artm = artm.ARTM(num_topics=T, topic_names=topic_names,
                           class_ids={"text": 1},#"title": 3,"desc": 1.5,"lang": 0}, #"keys": 0.5,
                           num_document_passes=2, reuse_theta=reuse_theta, cache_theta=cache_theta, seed=-1,
                           num_processors = 6)

    model_artm.scores.add(artm.PerplexityScore(name='PerplexityScore',topic_names=topic_names,
                                               use_unigram_document_model=False,dictionary='dictionary'))
    
    model_artm.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore',
                                                class_id="text",topic_names=topic_names))
    model_artm.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore',topic_names=topic_names))
    
    model_artm.scores.add(artm.TopTokensScore(name="top_words", num_tokens=15, class_id="text"))
    
# добавлеие в модель регуляризаторов
def setRegularizers(d=None,t=None,f=None,xx=None):
    #model_artm.regularizers.data.clear()
    #print d,t,f,xx
    global reguls
    reguls = [d,t,f]
    if d != None:
        len_d = len(d) if type(d) == tuple else 1
        if len_d == 1: d = [d]
        if d[0] != None:
            model_artm.regularizers.add(artm.DecorrelatorPhiRegularizer(name='DecorrPhi', tau=d[0]))
    if t != None:
        len_t = len(t) if type(t) == tuple else 1
        if len_t == 1: t = [t]
        if t[0] != None:
            model_artm.regularizers.add(artm.SmoothSparseThetaRegularizer(name='SparseTheta', tau=t[0]))

    if f != None:
        len_f = len(f) if type(f) == tuple else 1
        if len_f == 1: f = [f]
        if f[0] != None:
            model_artm.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SparsePhi', tau=f[0]))

# обучение модели и вывод статистики (метрик) по ней
def doFit(num_pass=2,doStat=True):
    t0 = time.time()
    model_artm.fit_offline(batch_vectorizer=batch_vectorizer,num_collection_passes=num_pass)
    global perpl
    perpl = [model_artm.score_tracker["PerplexityScore"].value[-1],
    model_artm.score_tracker['SparsityPhiScore'].value[-1],
    model_artm.score_tracker['SparsityThetaScore'].value[-1]]
    if doStat:
        print perpl, (time.time()-t0)/60
        getTopics("top_words",10,1)
        print '---'

# печать топ слов по 1-м 10 темам
def getTopics(name="top_words",num_top=15,step=1):
    n = 0
    for topic_name in model_artm.topic_names[::step]:
        print topic_name + ': ',
        tokens = model_artm.score_tracker[name].last_tokens
        for word in tokens[topic_name]:    
            print word,
        print
        n += 1
        if n == num_top:
            break

# график изменения метрики
def graphStat():
    plt.plot(model_artm.score_tracker["PerplexityScore"].value)

In [3]:
import time
import numpy as np

def saveFout(ii=-1,i=-1):
    fout.write('{0},{1},{2},{3},{4},{5},{6}'.format(ii,i,jj,x1,x2,x3,(time.time()-t0)/60.))
    if koef_min != None:
        fout.write((',{:.7}'*3).format(*koef_min))
    fout.write('\n')
    fout.flush()

doBreak = True
min_df = 1
doInitDic(model_path+name,min_df)
for jj in [100,200,300]:
    t0 = time.time()
    topics, num_bg, count_steps, perpl_min, x1, x2, x3, lExit, koef_min =\
    jj, 0, [24,16,24,20], None, None, None, None, False, None
    print '###',jj
    fout = open(model_path+name+'_'+str(topics)+'_'+str(min_df)+'.cfg','w')
    for ii in range(3):
        for i in range(len(count_steps[:3])):
            if i == 0:
                xx = [5e4,.1,0.0001][ii]
                vals_range = [xx*2**yy for yy in range(count_steps[0])]
            else:
                if dt[0] == None: dt[0] = dt[1]*0.9
                dy = (dt[2] - dt[0])/float(count_steps[i])
                vals_range = np.arange(dt[0]+dy*0.2, dt[2]-dy*0.2, dy)
            perpl0, dt, lFindMin, up = None, [None]*3, False, 0
            for z in vals_range:
                doInit(topics)
                model_artm.initialize(dictionary=dictionary)
                doFit(1,False)
                if ii == 0: setRegularizers(z)
                elif ii == 1: setRegularizers(x1,-z)
                elif ii == 2: setRegularizers(x1,x2,-z)
                doFit(3,False)
                print reguls,'%8.3f (%4.3f %4.3f)' % tuple(perpl)
                if perpl0 != None and perpl[0] > perpl0:
                    perpl0 = perpl[0]
                    #print '<<', up, i
                    if doBreak and ((ii < 3 and i == 0) or (up > 0 and z > x0)):
                        if i == 0:
                            dt[2] = z
                        break
                    else:
                        up += 1
                else:
                    perpl0, up = perpl[0], 0
                    if perpl_min == None or perpl_min > perpl0:
                        x0 = z
                        dt = [z-dy,z,z+dy] if i > 0 else [dt[1],z,z]
                        lFindMin = True
                        perpl_min = perpl0
                        koef_min = perpl
                        if ii == 0: x1 = z
                        elif ii == 1: x2 = -z
                        elif ii == 2: x3 = -z
            tt = (time.time()-t0)/60.
            print '---',(x1,x2,x3),perpl_min,lFindMin,dt,tt
            saveFout(ii,i)
            if dt[1] == None:
                if i == 0:
                    lExit = True
                else:
                    dt = [x0-dy,x0,x0+dy]
                    print '***',dt
            if lExit: break
        print '>>>',jj,ii,i,(time.time()-t0)/60.
        getTopics()
        if lExit: break
    print '===',jj,(x1,x2,x3),perpl_min,lFindMin,dt,lExit,(time.time()-t0)/60.
    saveFout()
    fout.close()

    print min_df, topics, x1,x2,x3

    for x in [x3]:
        t0 = time.time()
        doInit(topics)
        model_artm.initialize(dictionary=dictionary)
        doFit(1,False)
        #print topics,'%8.3f (%4.3f %4.3f)' % tuple(perpl), (time.time()-t0)/60.
        setRegularizers(x1,x2,x)
        doFit(15)
        print topics,reguls,'%8.3f (%4.3f %4.3f)' % tuple(perpl), (time.time()-t0)/60.

    model_artm.save(model_path+'model'+str(topics)+'_'+str(min_df)+'.dat')

loaded dict: models/booksX2/booksX2_1.dict!
### 100
[50000.0, None, None] 3066.665 (0.000 0.000)
[100000.0, None, None] 2832.680 (0.000 0.003)
[200000.0, None, None] 2746.850 (0.001 0.004)
[400000.0, None, None] 2587.266 (0.002 0.007)
[800000.0, None, None] 2403.930 (0.004 0.011)
[1600000.0, None, None] 2360.424 (0.009 0.019)
[3200000.0, None, None] 2839.871 (0.133 0.064)
--- (1600000.0, None, None) 2360.42392317 True [800000.0, 1600000.0, 3200000.0] 1.3185820659
[830000.0, None, None] 2391.888 (0.004 0.011)
[980000.0, None, None] 2374.707 (0.005 0.013)
[1130000.0, None, None] 2344.921 (0.006 0.014)
[1280000.0, None, None] 2354.684 (0.006 0.015)
[1430000.0, None, None] 2346.048 (0.007 0.017)
[1580000.0, None, None] 2345.429 (0.008 0.019)
[1730000.0, None, None] 2410.187 (0.010 0.020)
[1880000.0, None, None] 2445.380 (0.012 0.023)
--- (1130000.0, None, None) 2344.92136312 True [980000.0, 1130000.0, 1280000.0] 2.83358686368
[982500.0, None, None] 2373.996 (0.005 0.013)
[995000.0, None, N

In [14]:
print perpl
getTopics("top_words",300)

[1791.893459384577, 0.882978468163642, 0.9601844883453539]
sbj0:  город маленький прямо принимать мост ворота житель золотой городской площадь отель серый турок крыша чума
sbj1:  листницкий ася бунчук лопахин симамура демка гораций комако хуандо вяземский сотник евгений куин лили мужик
sbj2:  маруха иннокентий беатрис гест баронесса охранник марина барон вильямисар пачо жюльетта заремба зотов хедебю девица
sbj3:  взять дерево нужно именно красный точно рядом грудь лес немного кровь двор быстро прямо долина
sbj4:  продолжать улыбаться роберт пилар джордан верить мария бывать мост паркер фернандо пабло смеяться запах альберт
sbj5:  олег репортер николаевич сара русанов костоглот вадим джиггс опухоль павел лафкадио жоан кэйси дамасо хагуд
sbj6:  щукарь ильич жеребец конюшня штокман тихон серый макар дед кузьма ежели всадник баярд фургон мул
sbj7:  ингунн арнвид тур родич ныне магнхильда берг турхильда нестеренко григорс эйрик ингебьерг ивар горница аудун
sbj8:  консул будденброк тоня перм